<a href="https://colab.research.google.com/github/rviral001/SentAnalyzer/blob/master/sentiment/DistilBERTSentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install nlp

     |████████████████████████████████| 890kB 4.5MB/s 
     |████████████████████████████████| 3.0MB 26.4MB/s 
     |████████████████████████████████| 1.1MB 47.7MB/s 
     |████████████████████████████████| 890kB 57.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=0036aa26b3b78e1ccc93edfa93e242655e8ac9ca73b773182874a7ced3239e36
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.7MB 4.6MB/s 
     |████████████████████████████████| 245kB 30.3MB/s 
     |████████████████████████████████| 17.3MB 203kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
from transformers import AdamW, Trainer, TrainingArguments
import torch
import numpy as np
from nlp import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", return_dict=True)
for param in model.base_model.parameters():
    param.requires_grad = False
model.to('cuda')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'])
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    evaluate_during_training=True,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/nlp/utils/py_utils.py:191: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return function(data_struct)


{'loss': 0.64347119140625, 'learning_rate': 5e-05, 'epoch': 0.3198976327575176, 'step': 500}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{'loss': 0.4470240478515625, 'learning_rate': 4.834765366820886e-05, 'epoch': 0.6397952655150352, 'step': 1000}



{'eval_loss': 0.38387095655441283, 'eval_accuracy': 0.83488, 'eval_f1': 0.8308335382345711, 'eval_precision': 0.851705595698202, 'eval_recall': 0.81096, 'epoch': 0.6397952655150352, 'step': 1000}
{'loss': 0.375654296875, 'learning_rate': 4.6695307336417715e-05, 'epoch': 0.9596928982725528, 'step': 1500}



...


{'eval_loss': 0.3137205288887024, 'eval_accuracy': 0.86688, 'eval_f1': 0.8666987102459345, 'eval_precision': 0.8678806353280925, 'eval_recall': 0.86552, 'epoch': 8.317338451695457, 'step': 13000}
{'loss': 0.328046875, 'learning_rate': 7.038995373430271e-06, 'epoch': 8.637236084452976, 'step': 13500}
{'loss': 0.3390068359375, 'learning_rate': 5.3866490416391284e-06, 'epoch': 8.957133717210493, 'step': 14000}



{'eval_loss': 0.31404466384887697, 'eval_accuracy': 0.86684, 'eval_f1': 0.8680382130257264, 'eval_precision': 0.8602970063644221, 'eval_recall': 0.87592, 'epoch': 8.957133717210493, 'step': 14000}



{'loss': 0.3236298828125, 'learning_rate': 3.7343027098479845e-06, 'epoch': 9.277031349968011, 'step': 14500}
{'loss': 0.32815625, 'learning_rate': 2.081956378056841e-06, 'epoch': 9.596928982725528, 'step': 15000}



{'eval_loss': 0.31504945063591006, 'eval_accuracy': 0.86628, 'eval_f1': 0.8681314346574099, 'eval_precision': 0.8562757762041865, 'eval_recall': 0.88032, 'epoch': 9.596928982725528, 'step': 15000}
{'loss': 0.3333564453125, 'learning_rate': 4.2961004626569735e-07, 'epoch': 9.916826615483046, 'step': 15500}




TrainOutput(global_step=15630, training_loss=0.35294955614203455)

In [ ]:
tokenizer.save_vocabulary(".")

('./vocab.txt',)

In [ ]:
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  7592,  1010,  2026,  3899,  2003, 10140,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
